# Submittin an FL Hub job

In [1]:
# add path
import sys
print(sys.executable)
print(sys.version)

NVFLARE_HOME=f"/home/hroth/Code2/nvflare/flhub"
HUB_EXAMPLE=f"{NVFLARE_HOME}/examples/flhub"

/home/hroth/Code2/nvflare/flhub/examples/flhub/nvflare_flhub/bin/python3.8
3.8.15 (default, Oct 12 2022, 19:15:16) 
[GCC 11.2.0]


In [2]:
# (optional) check environment
!python --version
!pip freeze | grep nvflare
!pip freeze | grep torch
!pip freeze | grep monai

Python 3.8.15
monai-nvflare==0.2.1
-e git+ssh://git@github.com/holgerroth/NVFlare.git@d8c643f8b09306ae80d313fd6e8e00f9fac7a7dd#egg=nvflare_nightly
pytorch-ignite==0.4.10
torch==1.13.0
monai==1.0.1
monai-nvflare==0.2.1


### Open admin api runner

In [3]:
import msgpack
from nvflare.fuel.hci.client.fl_admin_api_runner import FLAdminAPIRunner, api_command_wrapper

# Initialize the runner
runner = FLAdminAPIRunner(
    username="admin@nvflare.com",
    admin_dir=f"{HUB_EXAMPLE}/workspaces/t1_workspace/admin@nvflare.com",
    poc=False,
    debug=False,
)

### Submit job

In [4]:
job_folder=f"{HUB_EXAMPLE}/job"
api_command_wrapper(runner.api.submit_job(job_folder))

{'status': <APIStatus.SUCCESS: 'SUCCESS'>, 'details': {'message': 'Submitted job: 296989cf-d3c4-4348-9a75-8c30c01a02fd', 'job_id': '296989cf-d3c4-4348-9a75-8c30c01a02fd'}, 'raw': {'time': '2022-11-09 13:54:03.770632', 'data': [{'type': 'string', 'data': 'Submitted job: 296989cf-d3c4-4348-9a75-8c30c01a02fd'}, {'type': 'success', 'data': ''}], 'status': <APIStatus.SUCCESS: 'SUCCESS'>}}


{'status': <APIStatus.SUCCESS: 'SUCCESS'>,
 'details': {'message': 'Submitted job: 296989cf-d3c4-4348-9a75-8c30c01a02fd',
  'job_id': '296989cf-d3c4-4348-9a75-8c30c01a02fd'},
 'raw': {'time': '2022-11-09 13:54:03.770632',
  'data': [{'type': 'string',
    'data': 'Submitted job: 296989cf-d3c4-4348-9a75-8c30c01a02fd'},
   {'type': 'success', 'data': ''}],
  'status': <APIStatus.SUCCESS: 'SUCCESS'>}}

### List jobs

In [5]:
%%capture
jobs = api_command_wrapper(runner.api.list_jobs())

In [6]:
print(jobs["raw"]["data"][0]["data"])

----------------------------------------------------------------------------------------------------------------------------
| JOB ID                               | NAME          | STATUS        | SUBMIT TIME                      | RUN DURATION   |
----------------------------------------------------------------------------------------------------------------------------
| d8e37db1-ede2-481d-8c3d-cf1bad4eec83 | spleen-bundle | FAILED_TO_RUN | 2022-11-09T13:48:06.143170-05:00 | N/A            |
| f899e659-a4c2-4923-84b2-bc75be8e4244 | spleen-bundle | RUNNING       | 2022-11-09T13:52:50.776034-05:00 | 0:01:11.338604 |
| 296989cf-d3c4-4348-9a75-8c30c01a02fd | spleen-bundle | SUBMITTED     | 2022-11-09T13:54:03.781260-05:00 | N/A            |
----------------------------------------------------------------------------------------------------------------------------



### Show server log

In [7]:
%%capture
reply = api_command_wrapper(runner.api.tail_target_log("server"))

In [8]:
print(reply["details"]["message"])

2022-11-09 13:53:34,929 - FederatedServer - INFO - received update from t1_project_t1_client_a_0 (1047 Bytes, 1668020014 seconds)
2022-11-09 13:53:37,762 - FederatedServer - INFO - received update from t1_project_t1_client_b_0 (1047 Bytes, 1668020017 seconds)
2022-11-09 13:53:48,612 - FederatedServer - INFO - GetTask: Return task: train to client: t1_client_b (db186635-f3bd-436f-a0ae-f4e85b444ac9) 
2022-11-09 13:53:48,958 - FederatedServer - INFO - GetTask: Return task: train to client: t1_client_a (16ff0c23-8836-4c52-8f6a-92818e53e93b) 
2022-11-09 13:53:52,291 - FederatedServer - INFO - received update from t1_project_t1_client_b_0 (1047 Bytes, 1668020032 seconds)
2022-11-09 13:53:54,206 - FederatedServer - INFO - received update from t1_project_t1_client_a_0 (1047 Bytes, 1668020034 seconds)
2022-11-09 13:53:56,676 - FederatedServer - INFO - GetTask: Return task: train to client: t1_client_a (16ff0c23-8836-4c52-8f6a-92818e53e93b) 
2022-11-09 13:53:57,038 - FederatedServer - INFO - Get

### Show Connector-A log

In [9]:
%%capture
reply = api_command_wrapper(runner.api.tail_target_log("t1_client_a"))

In [10]:
print(reply["details"]["message"])

    err = engine.deploy_app(
  File "/home/hroth/Code2/nvflare/flhub/nvflare/private/fed/client/client_engine.py", line 238, in deploy_app
    app_deployer = self.get_component(SystemComponents.APP_DEPLOYER)
AttributeError: type object 'SystemComponents' has no attribute 'APP_DEPLOYER'

2022-11-09 13:52:07,785 - FederatedClient - INFO - Got the new primary SP: localhost:8002
2022-11-09 13:52:08,843 - FederatedClient - INFO - Successfully registered client:t1_client_a for project t1_project. Token:16ff0c23-8836-4c52-8f6a-92818e53e93b SSID:ebc6125d-0a56-4688-9b08-355fe9e4d61a
2022-11-09 13:52:52,335 - ClientEngine - INFO - Starting client app. rank: 0
2022-11-09 13:52:52,355 - ProcessExecutor - INFO - Worker child process ID: 185135
2022-11-09 13:52:54,361 - ProcessExecutor - INFO - run (f899e659-a4c2-4923-84b2-bc75be8e4244): waiting for child worker process to finish.


### Show Connector-B log

In [11]:
%%capture
reply = api_command_wrapper(runner.api.tail_target_log("t1_client_b"))

In [12]:
print(reply["details"]["message"])

    err = engine.deploy_app(
  File "/home/hroth/Code2/nvflare/flhub/nvflare/private/fed/client/client_engine.py", line 238, in deploy_app
    app_deployer = self.get_component(SystemComponents.APP_DEPLOYER)
AttributeError: type object 'SystemComponents' has no attribute 'APP_DEPLOYER'

2022-11-09 13:52:07,803 - FederatedClient - INFO - Got the new primary SP: localhost:8002
2022-11-09 13:52:08,846 - FederatedClient - INFO - Successfully registered client:t1_client_b for project t1_project. Token:db186635-f3bd-436f-a0ae-f4e85b444ac9 SSID:ebc6125d-0a56-4688-9b08-355fe9e4d61a
2022-11-09 13:52:52,376 - ClientEngine - INFO - Starting client app. rank: 0
2022-11-09 13:52:52,395 - ProcessExecutor - INFO - Worker child process ID: 185143
2022-11-09 13:52:54,399 - ProcessExecutor - INFO - run (f899e659-a4c2-4923-84b2-bc75be8e4244): waiting for child worker process to finish.


### Finish

In [13]:
runner.api.logout()

{'status': <APIStatus.ERROR_INACTIVE_SESSION: 'ERROR_INACTIVE_SESSION'>,
 'details': 'session is logging out'}

# Show TensorBoard

In [14]:
# load only the hub server workspace
%load_ext tensorboard
%tensorboard --logdir "/home/hroth/Code2/nvflare/flhub/examples/flhub/workspaces/t1_workspace/localhost"

Note. The tensorboard should also be available at [http://localhost:6006](http://localhost:6006).